# 🚀 Arctic-Text2SQL-R1 Kaggle GPU 微調實戰

## 📋 專案概述

本筆記本實現了基於 **Snowflake Arctic-Text2SQL-R1 7B** 模型的 Text2SQL RAG 系統，專門針對 **Kaggle GPU 環境** 優化。

### 🎯 核心特色
- **模型**: Arctic-Text2SQL-R1 7B（SOTA 小模型，BIRD ExecAcc 57%）
- **微調方法**: QLoRA 4-bit 量化 + LoRA 適配器
- **記憶體需求**: ≈5GB VRAM（適合 Kaggle P100/T4）
- **斷點續訓**: 完整的檢查點管理系統
- **資料集**: SynSQL-2.5M 高質量合成資料

### 📊 預期效果
- **Spider-Dev**: EM 72% / Exec Acc 86%
- **BIRD-Dev**: Exec Acc ~57%
- **訓練時間**: 約 4-6 小時（Kaggle P100）

### 🔧 技術架構
```
自然語言查詢 → Schema檢索 → Arctic生成 → SQL執行驗證 → 自反饋修正
```

## 🛠️ 第一部分：環境設置與依賴安裝

### 重要提醒
- 確保 Kaggle 筆記本已啟用 **GPU** 加速器
- 建議使用 **P100** 或 **T4** GPU
- 預估訓練時間：4-6 小時

In [1]:
# 檢查 GPU 可用性與規格
import torch
import subprocess
import sys

print("🔍 系統環境檢查")
print("=" * 50)

# GPU 檢查
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU 可用: {gpu_name}")
    print(f"📊 GPU 記憶體: {gpu_memory:.1f} GB")
    
    # 檢查是否適合訓練
    if gpu_memory >= 15:
        print("🎯 記憶體充足，適合 7B 模型微調")
    elif gpu_memory >= 8:
        print("⚠️  記憶體中等，建議使用最激進的量化設置")
    else:
        print("❌ 記憶體不足，建議使用更小的模型")
else:
    print("❌ 未檢測到 GPU，請啟用 GPU 加速器")
    
# Python 版本
print(f"🐍 Python 版本: {sys.version.split()[0]}")

# Kaggle 環境檢查
import os
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    print(f"🏆 Kaggle 環境: {os.environ['KAGGLE_KERNEL_RUN_TYPE']}")
else:
    print("💻 本地環境")

🔍 系統環境檢查
✅ GPU 可用: Tesla P100-PCIE-16GB
📊 GPU 記憶體: 15.9 GB
🎯 記憶體充足，適合 7B 模型微調
🐍 Python 版本: 3.11.13
🏆 Kaggle 環境: Interactive


In [2]:
# 安裝必要套件 - 針對 Kaggle 環境優化
print("📦 安裝必要套件...")
print("=" * 50)

# 核心套件列表
packages = [
    "transformers>=4.36.0",  # 支援 Arctic 模型
    "peft>=0.7.0",           # PEFT/LoRA 支援
    "bitsandbytes>=0.41.0",  # QLoRA 量化
    "accelerate>=0.25.0",    # 分散式訓練
    "datasets>=2.15.0",      # 資料集處理
    "tensorboard",           # 訓練監控
    "wandb",                 # 實驗追蹤（可選）
    "sqlparse",              # SQL 解析
    "evaluate",              # 評估指標
    "scikit-learn",          # 機器學習工具
    "matplotlib",            # 視覺化
    "seaborn",               # 進階視覺化
    "tqdm"                   # 進度條
]

# 批量安裝
for package in packages:
    try:
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "-q", package],
            capture_output=True, text=True, timeout=300
        )
        if result.returncode == 0:
            print(f"✅ {package.split('>=')[0]} 安裝成功")
        else:
            print(f"⚠️  {package.split('>=')[0]} 安裝警告: {result.stderr[:100]}")
    except subprocess.TimeoutExpired:
        print(f"⏰ {package.split('>=')[0]} 安裝超時，跳過")
    except Exception as e:
        print(f"❌ {package.split('>=')[0]} 安裝失敗: {str(e)[:100]}")

print("\n🎉 套件安裝完成！")

📦 安裝必要套件...
✅ transformers 安裝成功
✅ peft 安裝成功
✅ bitsandbytes 安裝成功
✅ accelerate 安裝成功
✅ datasets 安裝成功
✅ tensorboard 安裝成功
✅ wandb 安裝成功
✅ sqlparse 安裝成功
✅ evaluate 安裝成功
✅ scikit-learn 安裝成功
✅ matplotlib 安裝成功
✅ seaborn 安裝成功
✅ tqdm 安裝成功

🎉 套件安裝完成！


In [ ]:
# 🔐 HuggingFace 認證設置 - Kaggle 優化版本
import getpass
import os
from huggingface_hub import login, whoami
from huggingface_hub.utils import HfHubHTTPError
import time

def setup_huggingface_auth_kaggle():
    """
    Kaggle 優化的 HuggingFace 認證設置
    更好的互動體驗和錯誤處理
    """
    
    print("🔐 HuggingFace 認證設置")
    print("=" * 50)
    
    # 檢查是否已經登入
    try:
        user_info = whoami()
        if user_info:
            print(f"✅ 已登入 HuggingFace")
            print(f"   • 用戶名: {user_info.get('name', 'Unknown')}")
            return True
    except:
        pass
    
    print("💡 建議使用 HuggingFace Token 以獲得：")
    print("   • 更快的下載速度")
    print("   • 避免 API 速率限制")
    print("   • 更穩定的連接")
    print()
    
    # 提供更明確的選擇
    print("請在下方選擇操作：")
    print("─" * 30)
    
    return True

def input_hf_token():
    """專門用於輸入 HuggingFace Token 的函數"""
    try:
        print("🔑 請輸入您的 HuggingFace Token：")
        print("💡 Token 不會顯示在螢幕上，請放心輸入")
        print("📝 如需獲取 Token：https://huggingface.co/settings/tokens")
        print()
        
        # 安全輸入
        hf_token = getpass.getpass("請輸入 Token: ").strip()
        
        if not hf_token:
            print("❌ Token 不能為空")
            return False
        
        print("🔄 正在驗證 Token...")
        
        # 登入驗證
        login(token=hf_token, add_to_git_credential=False)
        
        # 獲取用戶信息
        user_info = whoami()
        
        print("✅ HuggingFace 認證成功！")
        print(f"   • 用戶名: {user_info.get('name', 'Unknown')}")
        print(f"   • 用戶類型: {user_info.get('type', 'user')}")
        print(f"   • 認證狀態: 已驗證")
        
        return True
        
    except HfHubHTTPError as e:
        if e.response.status_code == 401:
            print("❌ Token 無效或已過期")
            print("   請檢查 Token 是否正確複製")
        else:
            print(f"❌ 網路錯誤: {e}")
        return False
        
    except Exception as e:
        print(f"❌ 認證失敗: {str(e)}")
        return False

def skip_auth():
    """跳過認證，使用匿名訪問"""
    print("🔓 使用匿名訪問")
    print("⚠️  注意：可能會遇到下載速度限制")
    print("   • 模型和資料集仍可正常使用")
    print("   • 如遇到問題，請重新運行並選擇認證")
    return False

# 執行初始設置
auth_setup_result = setup_huggingface_auth_kaggle()

print("\n" + "="*50)
print("🎯 HuggingFace 設置準備完成")
print("📋 接下來請選擇認證方式：")
print()
print("選項 A: 使用 Token（推薦）")
print("選項 B: 跳過認證")
print()
print("💡 請運行下方對應的 Cell 來完成設置")

In [ ]:
# 選項 B: 跳過認證，使用匿名訪問 🔓
# 如果您沒有 Token 或想跳過認證，請運行此 Cell

print("🔓 選項 B: 跳過認證")
print("=" * 40)

# 執行跳過認證
skip_result = skip_auth()

print("\n✅ 匿名訪問設置完成")
print("📝 注意事項：")
print("   • 下載速度可能較慢")
print("   • 可能遇到 API 限制")
print("   • 所有公開資源仍可正常使用")
print("\n🎯 可以繼續執行後續 Cell")

In [ ]:
# 選項 A: 使用 HuggingFace Token 認證 🔑
# 如果您有 HuggingFace Token，請運行此 Cell

print("🔑 選項 A: 使用 HuggingFace Token")
print("=" * 40)

# 執行 Token 輸入
auth_success = input_hf_token()

if auth_success:
    print("\n🎉 Token 認證完成！")
    print("✅ 可以享受更快的下載速度")
else:
    print("\n❌ Token 認證失敗")
    print("💡 您可以：")
    print("   1. 檢查 Token 是否正確")
    print("   2. 重新運行此 Cell")
    print("   3. 或運行下方 '選項 B' 使用匿名訪問")

## 🔄 第二部分：Kaggle 檢查點管理系統

### 核心設計理念
基於 README.md 建議，我們採用 **HuggingFace Trainer + PEFT** 架構：
- **成熟的斷點機制**: 內建 `resume_from_checkpoint` 功能
- **自動斷點偵測**: 使用 `trainer_utils.get_last_checkpoint()`
- **記憶體效率**: QLoRA 4-bit 量化讓 7B 模型僅需約 5GB VRAM
- **保存簡潔**: 只需保存 adapter 權重（通常 < 50MB）

In [ ]:
# 導入必要的庫
import os
import json
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# HuggingFace 生態系統
import transformers  # 添加這行來修復 transformers.__version__ 錯誤
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    TrainingArguments, 
    Trainer,
    BitsAndBytesConfig,
    EarlyStoppingCallback,
    TrainerCallback
)
from peft import LoraConfig, get_peft_model, PeftModel, TaskType
from datasets import Dataset, load_dataset
import transformers.trainer_utils as trainer_utils

# 其他工具
import sqlparse
from tqdm.auto import tqdm
import sqlite3

# 設定隨機種子確保結果可重現
torch.manual_seed(42)
np.random.seed(42)

# 設定圖表樣式
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ 所有庫導入成功！")
print(f"🔧 PyTorch 版本: {torch.__version__}")
print(f"🤗 Transformers 版本: {transformers.__version__}")

# Kaggle 專用檢查點管理器 - 防止訓練中斷的關鍵組件
class KaggleCheckpointManager:
    """
    Kaggle 專用的檢查點管理器
    
    功能特色：
    1. 自動偵測最新檢查點
    2. 保存/載入訓練元數據
    3. 支援多檢查點備份
    4. Kaggle 環境適配
    """
    
    def __init__(self, base_dir="/kaggle/working"):
        """初始化檢查點管理器
        
        Args:
            base_dir: 基礎目錄，Kaggle 環境建議使用 /kaggle/working
        """
        self.base_dir = Path(base_dir)
        self.checkpoint_dir = self.base_dir / "checkpoints"
        self.checkpoint_dir.mkdir(exist_ok=True, parents=True)
        
        # 元數據文件路徑
        self.metadata_file = self.checkpoint_dir / "training_metadata.json"
        
        print(f"📁 檢查點目錄: {self.checkpoint_dir}")

    def save_metadata(self, current_step: int, epoch: float, 
                     loss_history: List[float], learning_rate: float):
        """保存訓練元數據 - 關鍵的訓練狀態追蹤
        
        這個方法確保即使 Kaggle 意外中斷，我們也能精確知道訓練進度
        """
        metadata = {
            "current_step": current_step,
            "epoch": epoch,
            "timestamp": datetime.now().isoformat(),
            "loss_history": loss_history[-20:],  # 保存最近20個loss值用於分析
            "learning_rate": learning_rate,
            "kaggle_session": os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'unknown'),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "none",
            "gpu_memory_used": torch.cuda.memory_allocated() / 1024**3 if torch.cuda.is_available() else 0
        }
        
        try:
            with open(self.metadata_file, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False)
            print(f"💾 元數據已保存 [Step: {current_step}, Loss: {loss_history[-1]:.4f}]")
        except Exception as e:
            print(f"⚠️  元數據保存失敗: {e}")
    
    def load_metadata(self) -> Optional[Dict]:
        """載入訓練元數據 - 用於續訓時的狀態恢復"""
        if self.metadata_file.exists():
            try:
                with open(self.metadata_file, 'r', encoding='utf-8') as f:
                    metadata = json.load(f)
                
                print(f"📊 載入元數據成功:")
                print(f"   • 步驟: {metadata['current_step']}")
                print(f"   • 輪數: {metadata['epoch']:.2f}")
                print(f"   • 最新Loss: {metadata['loss_history'][-1]:.4f}")
                print(f"   • 學習率: {metadata['learning_rate']:.2e}")
                print(f"   • 時間: {metadata['timestamp']}")
                
                return metadata
            except Exception as e:
                print(f"❌ 元數據載入失敗: {e}")
                return None
        
        print("📝 未找到元數據文件，將創建新的訓練記錄")
        return None
    
    def get_latest_checkpoint(self) -> Optional[str]:
        """獲取最新檢查點路徑 - 自動續訓的核心功能"""
        try:
            checkpoint_path = trainer_utils.get_last_checkpoint(str(self.checkpoint_dir))
            
            if checkpoint_path:
                # 驗證檢查點完整性  
                checkpoint_files = list(Path(checkpoint_path).glob("*"))
                essential_files = ["adapter_config.json", "adapter_model.safetensors"]
                
                missing_files = []
                for file in essential_files:
                    if not (Path(checkpoint_path) / file).exists():
                        missing_files.append(file)
                
                if missing_files:
                    print(f"⚠️  檢查點不完整，缺少文件: {missing_files}")
                    return None
                
                print(f"🔄 找到完整檢查點: {checkpoint_path}")
                print(f"📂 包含文件: {len(checkpoint_files)} 個")
                return checkpoint_path
            else:
                print("🆕 未找到檢查點，將開始新的訓練")
                return None
                
        except Exception as e:
            print(f"❌ 檢查點檢測失敗: {e}")
            return None

# 初始化檢查點管理器
checkpoint_manager = KaggleCheckpointManager()

print("\n🎯 檢查點管理器初始化完成！")
print("   • 支援自動續訓")
print("   • 完整元數據追蹤")
print("   • Kaggle 環境優化")

## 🎯 第三部分：Arctic-Text2SQL-R1 模型配置

### 模型選擇理由
根據 README.md 分析，**Arctic-Text2SQL-R1 7B** 是最佳選擇：
- 🏆 **SOTA 表現**: BIRD ExecAcc 57%，同級模型最佳
- 💰 **資源友好**: 4-bit QLoRA 僅需 ≈5GB VRAM
- 🚀 **專業訓練**: Execution-reward RL 訓練，專注正確率
- 📜 **開源授權**: Apache-2.0，商用友好

In [ ]:
# Arctic-Text2SQL-R1 模型配置類
class ArcticModelConfig:
    """
    Arctic-Text2SQL-R1 7B 模型配置管理
    
    基於 README.md 建議，實現 QLoRA 4-bit 量化配置
    確保在 Kaggle P100/T4 環境下穩定運行
    """
    
    def __init__(self):
        # 模型基本信息
        self.model_name = "Snowflake/snowflake-arctic-instruct"  # Arctic 基座模型
        self.model_type = "arctic-text2sql"
        self.max_length = 2048  # 適合 Text2SQL 任務的序列長度
        
        # QLoRA 4-bit 量化配置 - 關鍵的記憶體優化
        self.quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,                    # 啟用 4-bit 量化
            bnb_4bit_quant_type="nf4",           # 使用 NF4 量化類型（推薦）
            bnb_4bit_compute_dtype=torch.bfloat16, # 計算精度（Arctic 推薦 bfloat16）
            bnb_4bit_use_double_quant=True,       # 雙重量化進一步節省記憶體
            bnb_4bit_quant_storage="uint8"       # 存儲格式
        )
        
        # LoRA 配置 - 基於 Arctic 架構特點調整
        self.lora_config = LoraConfig(
            # 目標模組 - Arctic 使用 MHA 結構
            target_modules=[
                "q_proj",    # Query 投影
                "k_proj",    # Key 投影  
                "v_proj",    # Value 投影
                "o_proj",    # Output 投影
                "gate_proj", # Arctic MoE gate
                "up_proj",   # MLP up 投影
                "down_proj"  # MLP down 投影
            ],
            r=32,                    # LoRA rank（平衡效果與效率）
            lora_alpha=64,          # LoRA scaling（通常是 r 的 2 倍）
            lora_dropout=0.05,      # LoRA dropout（防止過擬合）
            bias="none",            # 不訓練 bias
            task_type=TaskType.CAUSAL_LM,  # 因果語言模型
            use_rslora=True,        # 使用 RSLoRA（穩定性改進）
            use_dora=False          # 暫不使用 DoRA（節省計算）
        )
        
        print("🎯 Arctic 模型配置初始化完成：")
        print(f"   • 基座模型: {self.model_name}")
        print(f"   • 量化: 4-bit NF4 + 雙重量化")
        print(f"   • LoRA: r={self.lora_config.r}, α={self.lora_config.lora_alpha}")
        print(f"   • 目標模組: {len(self.lora_config.target_modules)} 個")
    
    def estimate_memory_usage(self) -> Dict[str, float]:
        """估算記憶體使用量 - 幫助 Kaggle 用戶規劃資源"""
        # 基於經驗值估算（7B 模型）
        base_model_4bit = 4.2  # 4-bit 量化後的基座模型
        lora_adapters = 0.3    # LoRA 適配器權重
        optimizer_states = 0.6  # AdamW optimizer 狀態
        gradient_cache = 0.4   # 梯度緩存
        activation_cache = 0.8  # 啟動值緩存
        misc_overhead = 0.7    # 其他開銷
        
        total_estimated = (
            base_model_4bit + lora_adapters + 
            optimizer_states + gradient_cache + 
            activation_cache + misc_overhead
        )
        
        return {
            "base_model_4bit": base_model_4bit,
            "lora_adapters": lora_adapters,
            "optimizer_states": optimizer_states,
            "gradient_cache": gradient_cache,
            "activation_cache": activation_cache,
            "misc_overhead": misc_overhead,
            "total_estimated": total_estimated
        }
    
    def print_memory_breakdown(self):
        """打印詳細的記憶體使用分析"""
        memory_usage = self.estimate_memory_usage()
        
        print("\n💾 預估記憶體使用量分析（GB）：")
        print("=" * 40)
        for component, usage in memory_usage.items():
            if component != "total_estimated":
                percentage = (usage / memory_usage["total_estimated"]) * 100
                print(f"  {component:.<20} {usage:>6.1f} GB ({percentage:4.1f}%)")
        
        print("-" * 40)
        print(f"  {'總計':.<20} {memory_usage['total_estimated']:>6.1f} GB (100.0%)")
        
        # GPU 兼容性檢查
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            print(f"\n🖥️  當前 GPU 記憶體: {gpu_memory:.1f} GB")
            
            if memory_usage["total_estimated"] <= gpu_memory * 0.9:  # 留 10% 緩衝
                print("✅ 記憶體充足，可以開始訓練")
            elif memory_usage["total_estimated"] <= gpu_memory:
                print("⚠️  記憶體緊張，建議監控使用量")
            else:
                print("❌ 記憶體不足，考慮：")
                print("   • 減少 batch size")
                print("   • 減少 max_length")
                print("   • 使用更小的 LoRA rank")

# 初始化模型配置
model_config = ArcticModelConfig()
model_config.print_memory_breakdown()

In [ ]:
# Arctic 模型載入與初始化
class ArcticModelLoader:
    """Arctic-Text2SQL 模型載入器 - 支援斷點續訓"""
    
    def __init__(self, config: ArcticModelConfig, checkpoint_manager: KaggleCheckpointManager):
        self.config = config
        self.checkpoint_manager = checkpoint_manager
        self.model = None
        self.tokenizer = None
        
        print(f"🚀 Arctic 模型載入器初始化")
    
    def load_tokenizer(self):
        """載入分詞器 - Arctic 專用配置"""
        print("\n📝 載入 Arctic 分詞器...")
        
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.config.model_name,
                trust_remote_code=True,          # Arctic 需要自定義代碼
                use_fast=True,                   # 使用快速分詞器
                padding_side="left"              # Text2SQL 任務建議左填充
            )
            
            # 設置特殊 token（如果未設置）
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
                print("   • 設置 pad_token = eos_token")
                
            if self.tokenizer.chat_template is None:
                # 為 Text2SQL 任務設置自定義模板
                self.tokenizer.chat_template = (
                    "{% for message in messages %}"
                    "{% if message['role'] == 'user' %}"
                    "### 指令:\\n根據資料庫結構生成SQL查詢\\n\\n### 輸入:\\n{{ message['content'] }}\\n\\n### 回應:\\n"
                    "{% elif message['role'] == 'assistant' %}"
                    "{{ message['content'] }}{% if not loop.last %}\\n\\n{% endif %}"
                    "{% endif %}"
                    "{% endfor %}"
                )
                print("   • 設置 Text2SQL 聊天模板")
            
            print(f"✅ 分詞器載入成功")
            print(f"   • 詞彙表大小: {len(self.tokenizer):,}")
            print(f"   • 特殊 token: pad={self.tokenizer.pad_token}, eos={self.tokenizer.eos_token}")
            
        except Exception as e:
            print(f"❌ 分詞器載入失敗: {e}")
            raise
    
    def load_base_model(self):
        """載入基座模型 - 支援 4-bit 量化"""
        print("\n🤖 載入 Arctic 基座模型（4-bit 量化）...")
        
        try:
            # 清理 GPU 記憶體
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                initial_memory = torch.cuda.memory_allocated() / 1024**3
                print(f"   • 初始 GPU 記憶體: {initial_memory:.2f} GB")
            
            # 載入量化模型
            self.model = AutoModelForCausalLM.from_pretrained(
                self.config.model_name,
                quantization_config=self.config.quantization_config,
                device_map="auto",               # 自動設備映射
                trust_remote_code=True,          # Arctic 需要自定義代碼
                torch_dtype=torch.bfloat16,      # Arctic 推薦精度
                attn_implementation="flash_attention_2",  # 使用 Flash Attention（如果可用）
                low_cpu_mem_usage=True,          # 降低 CPU 記憶體使用
                cache_dir="/kaggle/working/model_cache"  # Kaggle 緩存目錄
            )
            
            # 檢查記憶體使用
            if torch.cuda.is_available():
                after_memory = torch.cuda.memory_allocated() / 1024**3
                model_memory = after_memory - initial_memory
                print(f"   • 模型記憶體使用: {model_memory:.2f} GB")
                print(f"   • 總計 GPU 記憶體: {after_memory:.2f} GB")
            
            # 啟用梯度檢查點（節省記憶體）
            self.model.gradient_checkpointing_enable()
            print("   • 啟用梯度檢查點")
            
            print(f"✅ 基座模型載入成功")
            print(f"   • 參數量: {sum(p.numel() for p in self.model.parameters()):,}")
            print(f"   • 可訓練參數: {sum(p.numel() for p in self.model.parameters() if p.requires_grad):,}")
            
        except Exception as e:
            print(f"❌ 基座模型載入失敗: {e}")
            raise
    
    def apply_lora(self, resume_from_checkpoint: Optional[str] = None):
        """應用 LoRA 適配器 - 支援續訓"""
        if resume_from_checkpoint:
            print(f"\n🔄 從檢查點載入 PEFT 模型: {resume_from_checkpoint}")
            try:
                self.model = PeftModel.from_pretrained(
                    self.model,
                    resume_from_checkpoint,
                    is_trainable=True  # 重要：確保可以繼續訓練
                )
                print("✅ PEFT 模型續訓載入成功")
            except Exception as e:
                print(f"❌ PEFT 模型載入失敗: {e}")
                print("   將創建新的 LoRA 適配器")
                self.model = get_peft_model(self.model, self.config.lora_config)
        else:
            print("\n🆕 創建新的 LoRA 適配器...")
            self.model = get_peft_model(self.model, self.config.lora_config)
            print("✅ LoRA 適配器創建成功")
        
        # 打印可訓練參數統計
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in self.model.parameters())
        
        print(f"\n📊 LoRA 參數統計:")
        print(f"   • 可訓練參數: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)")
        print(f"   • 總參數: {total_params:,}")
        print(f"   • 記憶體效率: {100*(1-trainable_params/total_params):.1f}% 節省")
    
    def initialize_model(self, resume_from_checkpoint: Optional[str] = None):
        """完整模型初始化流程"""
        print("🎯 開始 Arctic-Text2SQL 模型初始化流程\n")
        
        # 步驟 1: 載入分詞器
        self.load_tokenizer()
        
        # 步驟 2: 載入基座模型
        self.load_base_model()
        
        # 步驟 3: 應用 LoRA
        self.apply_lora(resume_from_checkpoint)
        
        print("\n🎉 Arctic 模型初始化完成！準備開始訓練")
        
        return self.model, self.tokenizer

# 檢查是否有可用的檢查點
checkpoint_path = checkpoint_manager.get_latest_checkpoint()
metadata = checkpoint_manager.load_metadata()

# 初始化模型載入器
model_loader = ArcticModelLoader(model_config, checkpoint_manager)

print("🔧 模型載入器準備就緒")
if checkpoint_path:
    print(f"   將從檢查點續訓: {checkpoint_path}")
else:
    print("   將開始新的訓練")

## 📊 第四部分：SynSQL-2.5M 資料處理

### 資料集選擇理由
根據 README.md 建議，我們使用 **SynSQL-2.5M** 資料集：
- 🎯 **高品質**: 250萬個高質量合成樣本
- 🌐 **廣覆蓋**: 覆蓋1.6萬個資料庫結構
- 🔄 **可擴展**: 支持自由取樣，適合不同訓練規模
- 📈 **SOTA基礎**: 現代Text2SQL模型的標準訓練資料

In [ ]:
# SynSQL-2.5M 資料處理器
class SynSQLDataProcessor:
    """處理 SynSQL-2.5M 資料集的專用類別
    
    功能特色：
    1. 智能取樣 - 根據 Kaggle 資源限制調整資料量
    2. 格式標準化 - 統一 Text2SQL 格式
    3. 質量過濾 - 移除低品質樣本
    4. Kaggle 最佳化 - 考慮訓練時間限制
    """
    
    def __init__(self, tokenizer: AutoTokenizer, max_samples: int = 50000):
        """初始化資料處理器
        
        Args:
            tokenizer: Arctic 分詞器
            max_samples: 最大樣本數（Kaggle 建議 50K 以內）
        """
        self.tokenizer = tokenizer
        self.max_samples = max_samples
        self.max_length = 2048  # Arctic 建議長度
        
        # Text2SQL 提示模板
        self.prompt_template = (
            "### 指令\n"
            "根據給定的資料庫結構和自然語言問題，生成對應的 SQL 查詢。\n\n"
            "### 資料庫結構\n"
            "{schema}\n\n"
            "### 問題\n"
            "{question}\n\n"
            "### SQL 查詢\n"
            "{sql}"
        )
        
        print(f"📊 SynSQL 資料處理器初始化")
        print(f"   • 最大樣本數: {max_samples:,}")
        print(f"   • 序列長度: {self.max_length}")
    
    def load_synsql_dataset(self, subset_size: Optional[int] = None) -> Dataset:
        """載入 SynSQL-2.5M 資料集
        
        Args:
            subset_size: 子集大小，None 表示使用 max_samples
        """
        print("\n🔄 載入 SynSQL-2.5M 資料集...")
        
        try:
            # 實際大小
            actual_size = subset_size or self.max_samples
            
            # 載入資料集（取樣以節省時間）
            dataset = load_dataset(
                "seeklhy/SynSQL-2.5M",
                split=f"train[:{actual_size}]",  # 只取前 N 個樣本
                cache_dir="/kaggle/working/dataset_cache"  # Kaggle 緩存目錄
            )
            
            print(f"✅ 資料集載入成功")
            print(f"   • 樣本數: {len(dataset):,}")
            print(f"   • 資料欄位: {list(dataset.features.keys())}")
            
            # 顯示資料集統計
            self._print_dataset_stats(dataset)
            
            return dataset
            
        except Exception as e:
            print(f"❌ 資料集載入失敗: {e}")
            print("🔄 使用備用資料集創建方法...")
            return self._create_fallback_dataset()
    
    def _create_fallback_dataset(self) -> Dataset:
        """創建備用資料集（如果 SynSQL 載入失敗）"""
        print("📝 創建備用 Text2SQL 資料集...")
        
        # 備用樣本數據
        fallback_data = [
            {
                "schema": "CREATE TABLE users (id INT, name VARCHAR(50), email VARCHAR(100));",
                "question": "查找所有用戶的姓名和郵箱",
                "sql": "SELECT name, email FROM users;"
            },
            {
                "schema": "CREATE TABLE products (id INT, name VARCHAR(100), price DECIMAL(10,2));",
                "question": "找出價格超過100的產品",
                "sql": "SELECT * FROM products WHERE price > 100;"
            },
            {
                "schema": "CREATE TABLE orders (id INT, user_id INT, total DECIMAL(10,2));",
                "question": "計算訂單總金額",
                "sql": "SELECT SUM(total) FROM orders;"
            }
        ] * (self.max_samples // 3)  # 重複以達到所需樣本數
        
        return Dataset.from_list(fallback_data[:self.max_samples])
    
    def _print_dataset_stats(self, dataset: Dataset):
        """打印資料集統計信息"""
        print(f"\n📈 資料集統計分析:")
        
        # 檢查必要欄位
        required_fields = ['schema', 'question', 'sql']
        available_fields = list(dataset.features.keys())
        
        print(f"   • 必要欄位檢查:")
        for field in required_fields:
            if field in available_fields:
                print(f"     ✅ {field}")
            else:
                # 尋找相似欄位
                similar_fields = [f for f in available_fields if field.lower() in f.lower()]
                if similar_fields:
                    print(f"     ⚠️  {field} (找到相似: {similar_fields})")
                else:
                    print(f"     ❌ {field} (缺失)")
        
        # 樣本長度分析
        if len(dataset) > 0:
            sample = dataset[0]
            if 'question' in sample:
                avg_question_len = np.mean([len(str(item.get('question', ''))) for item in dataset[:1000]])
                print(f"   • 平均問題長度: {avg_question_len:.1f} 字符")
            
            if 'sql' in sample:
                avg_sql_len = np.mean([len(str(item.get('sql', ''))) for item in dataset[:1000]])
                print(f"   • 平均SQL長度: {avg_sql_len:.1f} 字符")
    
    def preprocess_dataset(self, dataset: Dataset, train_split: float = 0.9) -> Tuple[Dataset, Dataset]:
        """預處理資料集並分割為訓練/驗證集
        
        Args:
            dataset: 原始資料集
            train_split: 訓練集比例
        """
        print(f"\n🔧 開始資料預處理...")
        
        # 1. 資料清理和格式化
        print("   • 資料清理中...")
        cleaned_dataset = dataset.map(
            self._clean_and_format,
            remove_columns=dataset.column_names,  # 移除原始欄位
            desc="清理資料"
        )
        
        # 2. 過濾無效樣本
        print("   • 過濾無效樣本...")
        valid_dataset = cleaned_dataset.filter(
            lambda x: len(x['formatted_text']) > 10 and len(x['formatted_text']) < self.max_length * 4
        )
        
        print(f"   • 過濾後樣本數: {len(valid_dataset):,} (原始: {len(dataset):,})")
        
        # 3. 分詞化
        print("   • 分詞化處理...")
        tokenized_dataset = valid_dataset.map(
            self._tokenize_function,
            batched=True,
            batch_size=1000,
            desc="分詞化"
        )
        
        # 4. 分割訓練/驗證集
        print(f"   • 分割資料集 (訓練:{train_split:.0%}, 驗證:{1-train_split:.0%})...")
        split_dataset = tokenized_dataset.train_test_split(
            test_size=1-train_split,
            shuffle=True,
            seed=42
        )
        
        train_dataset = split_dataset['train']
        eval_dataset = split_dataset['test']
        
        print(f"✅ 資料預處理完成")
        print(f"   • 訓練集: {len(train_dataset):,} 樣本")
        print(f"   • 驗證集: {len(eval_dataset):,} 樣本")
        
        return train_dataset, eval_dataset
    
    def _clean_and_format(self, example: Dict) -> Dict:
        """清理並格式化單個樣本"""
        # 提取欄位（處理不同的欄位名稱）
        schema = example.get('schema', example.get('db_schema', ''))
        question = example.get('question', example.get('nl_question', example.get('query', '')))
        sql = example.get('sql', example.get('sql_query', ''))
        
        # 清理文本
        schema = str(schema).strip()
        question = str(question).strip()
        sql = str(sql).strip()
        
        # 格式化為統一的提示格式
        formatted_text = self.prompt_template.format(
            schema=schema,
            question=question,
            sql=sql
        )
        
        return {
            'formatted_text': formatted_text,
            'schema': schema,
            'question': question,
            'sql': sql
        }
    
    def _tokenize_function(self, examples: Dict) -> Dict:
        """批量分詞化函數"""
        # 分詞化
        tokenized = self.tokenizer(
            examples['formatted_text'],
            truncation=True,
            padding=False,  # 動態填充更有效
            max_length=self.max_length,
            return_tensors=None  # 返回 Python list
        )
        
        # 對於因果語言模型，labels 就是 input_ids
        tokenized['labels'] = tokenized['input_ids'].copy()
        
        return tokenized
    
    def create_data_collator(self):
        """創建資料整理器 - 用於動態填充"""
        from transformers import DataCollatorForLanguageModeling
        
        return DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,  # 不使用 MLM（因為是因果語言模型）
            pad_to_multiple_of=8  # 填充到8的倍數（提高效率）
        )

# 資料處理器將在模型載入後初始化
print("📊 SynSQL 資料處理器類別定義完成")
print("   等待模型載入後進行資料處理...")

## 🚀 第五部分：QLoRA 微調訓練實現

### 核心訓練架構
基於 README.md 最佳實踐，實現完整的 QLoRA 訓練流程：
- **HuggingFace Trainer**: 成熟的斷點續訓機制
- **自定義回調**: 實時監控訓練狀態和記憶體使用
- **動態調整**: 根據 GPU 記憶體自動調整批次大小
- **多指標評估**: Loss、學習率、GPU 使用率等全方位監控

In [ ]:
# QLoRA 訓練器 - 完整的訓練流程實現
class ArcticQLoRATrainer:
    """
    Arctic-Text2SQL QLoRA 微調訓練器
    
    核心功能：
    1. 自動記憶體管理和批次大小調整
    2. 完整的斷點續訓支援
    3. 實時訓練監控和日誌記錄
    4. Kaggle 環境優化配置
    """
    
    def __init__(self, 
                 model,
                 tokenizer, 
                 checkpoint_manager: KaggleCheckpointManager,
                 config: ArcticModelConfig):
        """初始化訓練器
        
        Args:
            model: 已配置 LoRA 的 Arctic 模型
            tokenizer: Arctic 分詞器
            checkpoint_manager: 檢查點管理器
            config: 模型配置
        """
        self.model = model
        self.tokenizer = tokenizer
        self.checkpoint_manager = checkpoint_manager
        self.config = config
        
        # 訓練狀態追蹤
        self.training_history = {
            'train_loss': [],
            'eval_loss': [],
            'learning_rate': [],
            'gpu_memory': [],
            'timestamps': []
        }
        
        print("🎯 Arctic QLoRA 訓練器初始化完成")
    
    def create_training_arguments(self, 
                                batch_size: int = 1,
                                gradient_accumulation_steps: int = 8,
                                num_epochs: int = 3,
                                learning_rate: float = 2e-5) -> TrainingArguments:
        """創建訓練參數配置 - 針對 Kaggle 環境優化"""
        
        return TrainingArguments(
            # 基本設置
            output_dir=str(self.checkpoint_manager.checkpoint_dir),
            overwrite_output_dir=False,  # 保護現有檢查點
            run_name=f"arctic-text2sql-{datetime.now().strftime('%Y%m%d_%H%M')}",
            
            # 訓練配置
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            
            # 優化器設置
            learning_rate=learning_rate,
            weight_decay=0.01,
            adam_beta1=0.9,
            adam_beta2=0.999,
            adam_epsilon=1e-8,
            max_grad_norm=1.0,
            
            # 學習率調度
            lr_scheduler_type="cosine",
            warmup_ratio=0.03,  # 3% warmup
            
            # 檢查點設置（頻繁保存適應 Kaggle）
            save_strategy="steps",
            save_steps=50,  # 每50步保存一次
            save_total_limit=3,  # 保留最近3個檢查點
            
            # 評估設置
            evaluation_strategy="steps",
            eval_steps=50,
            eval_accumulation_steps=4,  # 減少記憶體使用
            
            # 早停和模型選擇
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            
            # 日誌設置
            logging_strategy="steps",
            logging_steps=10,
            logging_dir=str(self.checkpoint_manager.base_dir / "logs"),
            report_to=["tensorboard"],
            
            # 性能優化
            fp16=True,  # 混合精度訓練
            dataloader_drop_last=True,
            remove_unused_columns=False,
            group_by_length=True,  # 按長度分組提高效率
            
            # Kaggle 特定優化
            dataloader_num_workers=2,  # 適合 Kaggle CPU 核心數
            ignore_data_skip=True,  # 加速續訓
            save_safetensors=True,  # 使用更安全的格式
            
            # 記憶體優化
            gradient_checkpointing=True,
            optim="adamw_torch_fused",  # 更快的優化器（如果可用）
        )
    
    def create_custom_callbacks(self):
        """創建自定義回調函數 - 增強監控和自動化功能"""
        
        class ArcticTrainingCallback(TrainerCallback):
            """Arctic 專用訓練回調 - 集成檢查點管理和記憶體監控"""
            
            def __init__(self, checkpoint_manager, training_history):
                self.checkpoint_manager = checkpoint_manager
                self.training_history = training_history
                self.best_eval_loss = float('inf')
                
            def on_log(self, args, state, control, model=None, logs=None, **kwargs):
                """日誌記錄時的回調 - 記錄詳細訓練狀態"""
                if logs:
                    current_time = datetime.now().isoformat()
                    
                    # 記錄訓練指標
                    if "train_loss" in logs:
                        self.training_history['train_loss'].append(logs["train_loss"])
                    if "eval_loss" in logs:
                        self.training_history['eval_loss'].append(logs["eval_loss"])
                    if "learning_rate" in logs:
                        self.training_history['learning_rate'].append(logs["learning_rate"])
                    
                    # 記錄 GPU 記憶體使用
                    if torch.cuda.is_available():
                        gpu_memory = torch.cuda.memory_allocated() / 1024**3
                        self.training_history['gpu_memory'].append(gpu_memory)
                    
                    self.training_history['timestamps'].append(current_time)
                    
                    # 每50步保存詳細元數據
                    if state.global_step % 50 == 0 and len(self.training_history['train_loss']) > 0:
                        self.checkpoint_manager.save_metadata(
                            current_step=state.global_step,
                            epoch=state.epoch,
                            loss_history=self.training_history['train_loss'],
                            learning_rate=logs.get("learning_rate", 0)
                        )
            
            def on_evaluate(self, args, state, control, model=None, logs=None, **kwargs):
                """評估完成時的回調 - 檢查是否需要早停"""
                if logs and "eval_loss" in logs:
                    current_eval_loss = logs["eval_loss"]
                    
                    # 更新最佳評估結果
                    if current_eval_loss < self.best_eval_loss:
                        self.best_eval_loss = current_eval_loss
                        print(f"🎉 新的最佳評估結果: {current_eval_loss:.4f}")
                    
                    # 記憶體使用報告
                    if torch.cuda.is_available():
                        memory_used = torch.cuda.memory_allocated() / 1024**3
                        memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
                        print(f"💾 GPU 記憶體: {memory_used:.1f}/{memory_total:.1f} GB ({memory_used/memory_total*100:.1f}%)")
            
            def on_save(self, args, state, control, model=None, **kwargs):
                """檢查點保存時的回調"""
                print(f"💾 檢查點已保存 - Step: {state.global_step}, Epoch: {state.epoch:.2f}")
        
        # 回調函數列表
        callbacks = [
            ArcticTrainingCallback(self.checkpoint_manager, self.training_history),
            EarlyStoppingCallback(
                early_stopping_patience=3,  # 3次評估無改善則早停
                early_stopping_threshold=0.01  # 改善閾值
            )
        ]
        
        return callbacks
    
    def setup_data_collator(self):
        """設置資料整理器 - 動態填充優化"""
        from transformers import DataCollatorForLanguageModeling
        
        return DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,  # 因果語言模型
            pad_to_multiple_of=8,  # 對齊到8的倍數提高效率
            return_tensors="pt"
        )
    
    def estimate_optimal_batch_size(self) -> Tuple[int, int]:
        """智能估算最佳批次大小 - 根據 GPU 記憶體動態調整"""
        if not torch.cuda.is_available():
            return 1, 8  # CPU 環境的保守設置
        
        gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
        
        # 基於 GPU 記憶體的啟發式規則
        if gpu_memory_gb >= 24:  # RTX 4090, A100 等
            return 2, 4
        elif gpu_memory_gb >= 16:  # P100, T4, RTX 3080 等
            return 1, 8
        elif gpu_memory_gb >= 12:  # GTX 1080 Ti 等
            return 1, 12
        else:  # 更小的 GPU
            return 1, 16
    
    def train_model(self, 
                   train_dataset: Dataset, 
                   eval_dataset: Dataset,
                   resume_from_checkpoint: Optional[str] = None,
                   custom_training_args: Optional[Dict] = None) -> Any:
        """執行完整的 QLoRA 微調訓練"""
        
        print("🚀 開始 Arctic-Text2SQL QLoRA 微調訓練")
        print("=" * 60)
        
        # 1. 智能批次大小設置
        optimal_batch_size, optimal_grad_accum = self.estimate_optimal_batch_size()
        print(f"📊 優化配置:")
        print(f"   • 批次大小: {optimal_batch_size}")
        print(f"   • 梯度累積步數: {optimal_grad_accum}")
        print(f"   • 有效批次大小: {optimal_batch_size * optimal_grad_accum}")
        
        # 2. 創建訓練參數
        training_args_kwargs = {
            'batch_size': optimal_batch_size,
            'gradient_accumulation_steps': optimal_grad_accum,
        }
        if custom_training_args:
            training_args_kwargs.update(custom_training_args)
        
        training_args = self.create_training_arguments(**training_args_kwargs)
        
        # 3. 設置回調和資料整理器
        callbacks = self.create_custom_callbacks()
        data_collator = self.setup_data_collator()
        
        # 4. 創建 Trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=self.tokenizer,
            data_collator=data_collator,
            callbacks=callbacks,
        )
        
        # 5. 開始訓練（支援續訓）
        print(f"\n🎯 訓練開始")
        if resume_from_checkpoint:
            print(f"   🔄 從檢查點續訓: {resume_from_checkpoint}")
            train_result = trainer.train(resume_from_checkpoint=resume_from_checkpoint)
        else:
            print("   🆕 開始新的訓練")
            train_result = trainer.train()
        
        # 6. 保存最終模型
        final_model_dir = self.checkpoint_manager.base_dir / "final_model"
        trainer.save_model(str(final_model_dir))
        
        # 7. 訓練總結
        print(f"\n🎉 訓練完成！")
        print(f"   • 最終模型保存至: {final_model_dir}")
        print(f"   • 訓練步數: {train_result.global_step}")
        print(f"   • 最終損失: {train_result.training_loss:.4f}")
        
        return train_result, trainer
    
    def plot_training_history(self):
        """可視化訓練歷史 - 生成詳細的訓練報告"""
        if not self.training_history['train_loss']:
            print("⚠️  沒有訓練歷史數據可供可視化")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Arctic-Text2SQL QLoRA 訓練監控', fontsize=16, fontweight='bold')
        
        # 損失曲線
        axes[0, 0].plot(self.training_history['train_loss'], label='訓練損失', color='blue', alpha=0.7)
        if self.training_history['eval_loss']:
            axes[0, 0].plot(self.training_history['eval_loss'], label='驗證損失', color='red', alpha=0.7)
        axes[0, 0].set_title('損失函數變化')
        axes[0, 0].set_xlabel('步數')
        axes[0, 0].set_ylabel('損失值')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 學習率曲線
        if self.training_history['learning_rate']:
            axes[0, 1].plot(self.training_history['learning_rate'], color='green', alpha=0.7)
            axes[0, 1].set_title('學習率調度')
            axes[0, 1].set_xlabel('步數')
            axes[0, 1].set_ylabel('學習率')
            axes[0, 1].grid(True, alpha=0.3)
        
        # GPU 記憶體使用
        if self.training_history['gpu_memory']:
            axes[1, 0].plot(self.training_history['gpu_memory'], color='purple', alpha=0.7)
            axes[1, 0].set_title('GPU 記憶體使用')
            axes[1, 0].set_xlabel('步數')
            axes[1, 0].set_ylabel('記憶體 (GB)')
            axes[1, 0].grid(True, alpha=0.3)
        
        # 訓練進度統計
        axes[1, 1].axis('off')
        stats_text = f"""
訓練統計摘要:
        
• 總訓練步數: {len(self.training_history['train_loss'])}
• 最終訓練損失: {self.training_history['train_loss'][-1]:.4f}
• 最低驗證損失: {min(self.training_history['eval_loss']) if self.training_history['eval_loss'] else 'N/A'}
• 平均 GPU 記憶體: {np.mean(self.training_history['gpu_memory']):.1f} GB
• 訓練持續時間: {len(self.training_history['timestamps'])} 個記錄點
        """
        axes[1, 1].text(0.1, 0.9, stats_text, transform=axes[1, 1].transAxes, 
                        fontsize=12, verticalalignment='top', fontfamily='monospace')
        
        plt.tight_layout()
        plt.show()
        
        # 保存圖表
        plot_path = self.checkpoint_manager.base_dir / "training_history.png"
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"📊 訓練歷史圖表已保存: {plot_path}")

print("🔧 Arctic QLoRA 訓練器類別定義完成")
print("   準備開始模型初始化和資料載入...")

## 🎯 第六部分：完整訓練流程執行

### 執行步驟
1. **模型初始化** - 載入 Arctic 模型並配置 LoRA
2. **資料載入** - 處理 SynSQL 資料集並預處理
3. **訓練執行** - 開始 QLoRA 微調訓練
4. **結果評估** - 分析訓練結果和模型性能

In [ ]:
# 步驟 1: 初始化 Arctic 模型和分詞器
print("🚀 第一步：初始化 Arctic-Text2SQL 模型")
print("=" * 50)

try:
    # 初始化模型（支援斷點續訓）
    model, tokenizer = model_loader.initialize_model(checkpoint_path)
    
    print("\n✅ 模型初始化成功！")
    print(f"   • 模型類型: {type(model).__name__}")
    print(f"   • 分詞器類型: {type(tokenizer).__name__}")
    
except Exception as e:
    print(f"❌ 模型初始化失敗: {e}")
    print("請檢查:")
    print("   • GPU 記憶體是否充足")
    print("   • 網路連接是否正常")
    print("   • Hugging Face 是否可訪問")
    raise

In [ ]:
# 步驟 2: 載入和預處理 SynSQL 資料集
print("📊 第二步：載入和預處理 SynSQL-2.5M 資料集")
print("=" * 50)

try:
    # 初始化資料處理器
    data_processor = SynSQLDataProcessor(
        tokenizer=tokenizer,
        max_samples=10000  # Kaggle 環境建議樣本數，可根據需要調整
    )
    
    # 載入原始資料集
    print("\n🔄 載入原始資料集...")
    raw_dataset = data_processor.load_synsql_dataset()
    
    # 預處理和分割資料集
    print("\n🔧 預處理資料集...")
    train_dataset, eval_dataset = data_processor.preprocess_dataset(
        raw_dataset, 
        train_split=0.9  # 90% 用於訓練，10% 用於驗證
    )
    
    # 顯示樣本示例
    print("\n📋 資料集樣本預覽:")
    print("訓練集樣本:")
    sample_idx = 0
    if len(train_dataset) > 0:
        sample = train_dataset[sample_idx]
        print(f"  • Input IDs 長度: {len(sample['input_ids'])}")
        print(f"  • Labels 長度: {len(sample['labels'])}")
        
        # 解碼一個樣本看看格式
        decoded_text = tokenizer.decode(sample['input_ids'][:200], skip_special_tokens=True)
        print(f"  • 樣本內容預覽: {decoded_text[:200]}...")
    
    print(f"\n✅ 資料集準備完成！")
    print(f"   • 訓練集樣本數: {len(train_dataset):,}")
    print(f"   • 驗證集樣本數: {len(eval_dataset):,}")
    
except Exception as e:
    print(f"❌ 資料集載入失敗: {e}")
    print("   嘗試使用備用資料集...")
    
    # 使用備用資料集
    data_processor = SynSQLDataProcessor(tokenizer=tokenizer, max_samples=1000)
    raw_dataset = data_processor._create_fallback_dataset()
    train_dataset, eval_dataset = data_processor.preprocess_dataset(raw_dataset)
    
    print(f"✅ 備用資料集載入成功")
    print(f"   • 訓練集樣本數: {len(train_dataset):,}")
    print(f"   • 驗證集樣本數: {len(eval_dataset):,}")

In [ ]:
# 步驟 3: 初始化訓練器並開始 QLoRA 微調
print("🎯 第三步：開始 QLoRA 微調訓練")
print("=" * 50)

# 初始化 QLoRA 訓練器
trainer = ArcticQLoRATrainer(
    model=model,
    tokenizer=tokenizer,
    checkpoint_manager=checkpoint_manager,
    config=model_config
)

# 配置訓練參數（可根據需要調整）
custom_training_config = {
    'num_epochs': 2,  # Kaggle 時間限制，使用較少輪次
    'learning_rate': 2e-5,  # Arctic 推薦學習率
}

print("🚀 開始訓練...")
print("💡 提示：訓練過程中會自動保存檢查點，可以安全中斷並續訓")
print("\n" + "="*60)

try:
    # 開始訓練（支援續訓）
    train_result, trained_model = trainer.train_model(
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        resume_from_checkpoint=checkpoint_path,
        custom_training_args=custom_training_config
    )
    
    print("\n" + "="*60)
    print("🎉 訓練成功完成！")
    print(f"   • 總訓練步數: {train_result.global_step}")
    print(f"   • 最終訓練損失: {train_result.training_loss:.4f}")
    print(f"   • 訓練用時: {train_result.metrics.get('train_runtime', 0):.1f} 秒")
    
except KeyboardInterrupt:
    print("\n⏸️  訓練被用戶中斷")
    print("💾 檢查點已自動保存，可稍後續訓")
    
except Exception as e:
    print(f"\n❌ 訓練過程中出現錯誤: {e}")
    print("💾 檢查檢查點是否已保存")
    print("🔄 可嘗試從最新檢查點續訓")
    
    # 顯示可用的檢查點
    available_checkpoints = list(checkpoint_manager.checkpoint_dir.glob("checkpoint-*"))
    if available_checkpoints:
        print(f"📂 可用檢查點: {len(available_checkpoints)} 個")
        for cp in sorted(available_checkpoints)[-3:]:  # 顯示最近3個
            print(f"   • {cp.name}")
    
    raise

In [ ]:
# 步驟 4: 訓練結果分析和可視化
print("📊 第四步：訓練結果分析")
print("=" * 50)

# 生成訓練歷史可視化
try:
    print("🎨 生成訓練歷史圖表...")
    trainer.plot_training_history()
    
except Exception as e:
    print(f"⚠️  可視化生成失敗: {e}")
    print("   可能原因：沒有足夠的訓練數據")

# 顯示最終統計
print("\n📈 訓練總結統計:")
print("=" * 30)

try:
    history = trainer.training_history
    
    if history['train_loss']:
        print(f"✅ 訓練指標:")
        print(f"   • 總訓練步數: {len(history['train_loss'])}")
        print(f"   • 初始損失: {history['train_loss'][0]:.4f}")
        print(f"   • 最終損失: {history['train_loss'][-1]:.4f}")
        print(f"   • 損失改善: {history['train_loss'][0] - history['train_loss'][-1]:.4f}")
        
        if history['eval_loss']:
            print(f"   • 最佳驗證損失: {min(history['eval_loss']):.4f}")
        
        if history['gpu_memory']:
            print(f"   • 平均 GPU 使用: {np.mean(history['gpu_memory']):.1f} GB")
            print(f"   • 峰值 GPU 使用: {max(history['gpu_memory']):.1f} GB")
    
    # 檢查點信息
    print(f"\n💾 檢查點信息:")
    checkpoints = list(checkpoint_manager.checkpoint_dir.glob("checkpoint-*"))
    print(f"   • 保存的檢查點數: {len(checkpoints)}")
    
    final_model_path = checkpoint_manager.base_dir / "final_model"
    if final_model_path.exists():
        print(f"   • 最終模型路徑: {final_model_path}")
        model_files = list(final_model_path.glob("*"))
        print(f"   • 模型文件數: {len(model_files)}")
    
except Exception as e:
    print(f"⚠️  統計信息生成失敗: {e}")

print("\n🎊 Arctic-Text2SQL QLoRA 微調完成！")
print("=" * 50)
print("✅ 您已成功完成:")
print("   • Arctic-Text2SQL-R1 7B 模型微調")
print("   • QLoRA 4-bit 量化訓練")
print("   • SynSQL 資料集訓練")
print("   • 完整的檢查點管理")
print("   • 訓練過程監控")

print("\n🚀 下一步建議:")
print("   • 使用訓練好的模型進行 SQL 生成測試")
print("   • 在 BIRD/Spider 基準上評估模型性能")
print("   • 部署到生產環境進行實際應用")
print("   • 根據業務需求進一步微調")